# Module Import

In [ ]:
!pip install konlpy
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch
!pip install KRWordRank
!pip install soynlp

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
import pandas as pd
from konlpy.tag import Okt
from tqdm import tqdm, tqdm_notebook
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from collections import Counter
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Embedding, Dense, GRU, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow.keras.backend as K
from gensim.models import Word2Vec
import gensim
from konlpy.tag import Hannanum
from krwordrank.word import KRWordRank
from krwordrank.word import summarize_with_keywords
from soynlp.tokenizer import LTokenizer

In [ ]:
device = torch.device("cuda:0")

# Loading Data

In [ ]:
def dep_list_converter(deplist):
  con_list = []
  for dep in deplist:
    if '내과' in dep:
      if '내과' in con_list:
        continue
      con_list.append('내과')
    elif '외과' in dep:
      if '외과' in con_list:
        continue
      con_list.append('외과')
    else:
      con_list.append(dep)
  return con_list

def dep_string_converter(depstr):
  try:
    deplist = depstr.split(',')
    con_list = []
    for dep in deplist:
      if '내과' in dep:
        if '내과' in con_list:
          continue
        con_list.append('내과')
      elif '외과' in dep:
        if '외과' in con_list:
          continue
        con_list.append('외과')
      else:
        con_list.append(dep.lstrip())
    return ','.join(con_list)
  except:
    return None

In [ ]:
#데이터셋 가공
obj_data = pd.read_csv('/content/drive/MyDrive/Team01_KoBERT/symptom_dataset.csv')
# obj_data['department'] = obj_data['department'].apply(lambda x: dep_string_converter(x))
obj_data['symptom'] = obj_data['symptom'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "")
obj_data['name'] = obj_data['name'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "")
obj_data['simple_description'] = obj_data['simple_description'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "")
obj_data['symptom'] = obj_data['symptom'] + ' ' + obj_data['simple_symptom'] + ' ' + obj_data['site']
obj_result = obj_data['department'].str.replace(' ', '').str.split(',')
# obj_result = obj_result.apply(lambda x: dep_list_converter(x))
obj_result = obj_result.apply(lambda x: pd.Series(x))
obj_result = obj_result.stack().reset_index(level=1, drop=True).to_frame('department_single')
obj_result = obj_data.merge(obj_result, left_index=True, right_index=True, how='left')

<ipython-input-423-2a9a59a7d283>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  obj_data['symptom'] = obj_data['symptom'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "")
<ipython-input-423-2a9a59a7d283>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  obj_data['name'] = obj_data['name'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "")
<ipython-input-423-2a9a59a7d283>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  obj_data['simple_description'] = obj_data['simple_description'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "")


In [ ]:
sub_data = pd.read_csv('/content/drive/MyDrive/Team01_KoBERT/symptom_dataset_by_naver.csv')
sub_data['department'] = sub_data['department'].fillna('비뇨기과')
# sub_data['department'] = sub_data['department'].apply(lambda x: dep_string_converter(x))
sub_data['name'] = sub_data['name'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "")
sub_data['symptom'] = sub_data['symptom'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "")
sub_result = sub_data['department'].str.replace(' ', '').str.split(',')
# sub_result = sub_result.apply(lambda x: dep_list_converter(x))
sub_result = sub_result.apply(lambda x: pd.Series(x))
sub_result = sub_result.stack().reset_index(level=1, drop=True).to_frame('department_single')
sub_result = sub_data.merge(sub_result, left_index=True, right_index=True, how='left')
sub_result['symptom'] = sub_result['symptom'] + ' ' + sub_result['name']
sub_result['symptom'] = sub_result['symptom'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "")

<ipython-input-424-99fac2b99a46>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  sub_data['name'] = sub_data['name'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "")
<ipython-input-424-99fac2b99a46>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  sub_data['symptom'] = sub_data['symptom'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "")
<ipython-input-424-99fac2b99a46>:12: FutureWarning: The default value of regex will change from True to False in a future version.
  sub_result['symptom'] = sub_result['symptom'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "")


In [ ]:
data_result =  pd.concat([obj_result[['symptom', 'department_single']], sub_result[['symptom', 'department_single']]])

In [ ]:
data_result = data_result.dropna()

In [ ]:
data_result['department_single'].value_counts()

소아청소년과      378
피부과         182
이비인후과       178
신경과         177
소화기내과       164
산부인과        147
가정의학과       144
정신건강의학과     135
감염내과        130
정형외과        125
신경외과        116
외과          115
호흡기내과       112
혈액종양내과      107
안과           90
방사선종양학과      76
재활의학과        76
순환기내과        71
흉부외과         67
내분비내과        59
신장내과         58
비뇨기과         58
알레르기내과       39
응급의학과        37
류마티스내과       32
성형외과         23
치과           23
마취통증의학과      19
내과           13
진단검사의학과       4
병리과           2
핵의학과          2
신종플루거점병원      1
이식혈관외과        1
Name: department_single, dtype: int64

In [ ]:
obj_data.head()

,name,simple_description,department,site,simple_symptom,symptom
0,간흡충증,기생충인 간흡충이 쓸개즙이 내려오는 통로인 담관에 기생하면서 여러 가지 병을 일으키...,"감염내과, 소화기내과, 가정의학과","간, 담관",설사,담관에 있는 성충이 물리적 화학적 자극을 가하여 이상 증상이 나타난다 증상으로는 배...
1,감기,여 종 이상의 바이러스에 의해 발생하는 호흡기계의 감염 증상,"가정의학과, 감염내과, 호흡기내과, 소아청소년과, 이비인후과","상부 호흡기계, 코, 목","기침 , 인두통 및 인후통 , 비루 , 비폐색 , 재채기 , 근육통 , 발열",감기 바이러스에 노출된 지 일 후에 증상이 나타난다 증상은 감기 바이러스가 상부 호...
2,광견병,광견병 바이러스를 가지고 있는 동물에 사람이 물렸을 때 발생하는 급성 뇌척수염,"감염내과, 신경과","중추신경계(뇌, 척수)","근육간대경련 , 혼수 , 아린감 , 근육통",광견병 바이러스에 노출된 후 증상이 나타나는 데 걸리는 시간은 여러 가지 요인에 영...
3,한센병,나균에 의한 감염증으로 나균이 피부 말초 신경계 상부 기도를 침범하여 병적인 변화를...,"감염내과, 피부과","피부, 말초신경계, 상기도","비출혈 , 결절 , 구진 , 무감각증 , 아린감 , 근육통 , 홍반 , 반점",잠복기간은 개월년으로 다양하다 한센병은 피부에 나타나는 병적인 변화의 종류에 따라 ...
4,노로 바이러스,노로 바이러스 감염에 의한 유행성 바이러스성 위장관염,"감염내과, 소화기내과, 소아청소년과",위장관,"근육통 , 구역 , 설사 , 구토 , 복통 , 핍뇨 및 무뇨",사람이 노로 바이러스에 감염되면 평균 시간의 잠복기를 거친뒤에 갑자기 오심 구토 설...


In [ ]:
sub_data.head()

,Unnamed: 0,name,department,symptom
0,0,범혈구감소증,혈액종양내과,감소된 혈액세포의 종류와 정도에 따라 아래와 같은 증상이 일어날수 있다 백혈구감소 ...
1,1,반점치,치과,법랑질 저형성증은 부분적으로 또는 광범위하게 나타날 수 있다 저형성의 정도에 따라 ...
2,2,십이지장 용종,소화기내과,대개의 십이지장 용종은 그 증상이 없거나 크지 않은 경우가 대부분이지만 심한 소화성...
3,3,버킷림프종,혈액종양내과,악성 림프종의 흔한 증상은 통증 없이 림프절이 커지는 것이며 전신적으로 발열 야간 ...
4,4,골수섬유증,혈액종양내과,초기에는 증상이 보이지 않을수도 있다 섬유화가 진행이 되어 혈액세포가 감소하게 되면...


# Word2Vec Test

In [ ]:
from gensim.models.word2vec import Word2Vec

In [ ]:
train_data = data_result.groupby('department_single').sum()

In [ ]:
train_data

,symptom
department_single,
가정의학과,담관에 있는 성충이 물리적 화학적 자극을 가하여 이상 증상이 나타난다 증상으로는 배...
감염내과,담관에 있는 성충이 물리적 화학적 자극을 가하여 이상 증상이 나타난다 증상으로는 배...
내과,수족냉증의 증상은 추운 곳에 있을 때뿐만 아니라 따뜻한 곳에서도 손발이 시리듯 차다...
내분비내과,호르몬 분비 없이 세포 덩어리만을 형성하는 비기능성 종양은 주로 종괴 효과로 인하여...
류마티스내과,경피증은 만성 다기관 질환으로 초기 증상은 대개 비특이적이며 레이노 현상 추위나 ...
마취통증의학과,활동성 근막유발점통증 유발점은 골격근에 존재하는 과민감성 부위이며 극심한 압통을 나...
방사선종양학과,방광암의 가장 주된 증상은 통증 없이 소변에 피가 섞여 나오는 것이다 혈뇨의 정도는...
병리과,제자리암은 대부분 특별한 증상이 없이 우연히 발견되는 경우가 많다 자궁경부의 제자리...
비뇨기과,기 매독의 주요 증상은 통증이 없는 피부궤양이다 대부분의 경우 한 개의 궤양만 관찰...


In [ ]:
okt = Okt()
hannanum = Hannanum() 
stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다', '이다', ',', '"', "'", ';', '.', '/', '(', ')', '[', ']', '-', '_', '=', '증상', '있다', '경우', '나타', '한다', '발생', '등의', '등이', '따라', '있는', '된다', '가장', '주로', '심한', '동반', '또는', '이상', '환자', '같은', '것이', '의해', '다른', '대부분', '있으며', '의한', '정도', '부위', '대개', '흔히', '매우', '또한', '진행', '인한', '인해', '많다',' 않는', '보이', '이러한', '흔한', '때문에', '있고', '함께', '초기', '보인다', '혹은', '많이', '지속', '없는', '다양한', '보통', '원인', '특징', '특히', '대한', '여러', '하며', '그러나', '없이', '드물게', '이를', '서서히', '일부', '등을', '하고', '시작', '하지', '흔하', '하는', '것으로', '이는', '있을', '후에', '않고', '비교적', '되면', '가지', '되고', '쉽게', '사람', '일어', '일반', '등으로', '못하', '없다', '비해', '따른', '이후', '일으', '있어', '자주', '것을', '많은', '에서', '것은', '내에', '위치', '외에', '않은', '그리고', '더러', '간혹', '당황', '또', '또한', '더해', '더는', '더이상', '그리고', '느끼', '알려져', '동안', '되며', '듯한', '걸쳐', '달리', '어느', '있지만', '통해', '어려', '어지', '이런', '이로', '때는', '우연히', '아니라', '이내에', '전형적인', '되어', '들어', '않으면', '보일', '계속', '처음', '필요', '다르게', '않아', '때로는', '빠르게', '말한다', '어떤', '중에', '위해', '인하여', '종종', '문제가', '크게', '떨어', '지나', '인지', '많으며', '대표적인', '이와', '경향이', '변하', '있거나', '먼저', '높은', '이외에도', '다음', '급격히', '기타', '시에', '있습니다', '주어', '편이다', '단순', '전에', '자율', '증세가', '의하여', '비특이적', '아무런', '쪽으로', '과도한', '병원을', '비슷한', '이에', '등과', '반드시']

In [ ]:
token_data = data_result.drop_duplicates(['symptom'])
tokenized_data = []
for sentence in tqdm(token_data['symptom']):
    tmp = []
    tmp  = okt.morphs(sentence, stem=True) # 토큰화
    tmp  = [word for word in tmp if not word in stopwords] # 불용어 제거
    tmp  = [word  if word != '귀가' else '귀' for word in tmp]
    tokenized_data.append(tmp)

100%|██████████| 1808/1808 [01:34<00:00, 19.07it/s]


In [ ]:
model = Word2Vec(sentences = tokenized_data, size = 100, window = 20, min_count = 1, workers = 5, sg = 1)

In [ ]:
def data_tokenizer(sentence):
  tmp = []
  tokenized = okt.morphs(sentence, stem=True) # 토큰화
  stopwords_removed = [word for word in tokenized if not word in stopwords] # 불용어 제거
  changed = [word if word != '귀가' else '귀' for word in stopwords_removed]
  tmp.append(changed)
  return tmp

In [ ]:
train_data['symptom'] = train_data['symptom'].apply(lambda x: data_tokenizer(x)[0])

In [ ]:
train_data

In [ ]:
# train_data['symptom'] = train_data['symptom'].apply(lambda x: list(set(x)))

In [ ]:
import operator

In [ ]:
def test_predict(sentence):
  word_list = data_tokenizer(sentence)
  tmp = []
  result = {}
  for word in word_list[0]:
    tmp.append(word)
    try:
        tmp = tmp + [i for i, v in model.wv.most_similar(word)[:]]
    except:
        continue
  print(tmp)
  for i in train_data.index:
    result[i] = len(set(train_data['symptom'][i]).intersection(set(tmp)))
  return list(dict(sorted(result.items(), key=operator.itemgetter(1), reverse=True)).keys())


In [ ]:
model = Word2Vec(sentences = tokenized_data, size = 100, window = 20, min_count = 0, workers = 10, sg = 1)
model.wv.most_similar('귀')

[('귀의', 0.810023307800293),
 ('턱밑', 0.7846348881721497),
 ('하선', 0.7708806991577148),
 ('이충만', 0.7683533430099487),
 ('밑샘', 0.7607894539833069),
 ('이통', 0.7579001784324646),
 ('외다', 0.7569637298583984),
 ('이명', 0.7537832260131836),
 ('귓불', 0.7501367926597595),
 ('샘', 0.7423829436302185)]

In [ ]:
print(test_predict('두통이 심하고 어지러움'))

['두통', '메', '졸림', '슥', '구역질', '구역', '귀울림', '요하', '에서보다', '구근', '보다도', '심하다', '회복하다', '신체검사', '요하', '신속하다', '수주로', '오기', '역', '구근', '패혈', '늑', '어지러움', '교감', '공복', '고산병', '심박', '고산', '사하다', '식다', '심정지', '심박수', '기색']
['가정의학과', '감염내과', '방사선종양학과', '소화기내과', '순환기내과', '신장내과', '혈액종양내과', '내과', '소아청소년과', '신경과', '신경외과', '응급의학과', '정신건강의학과', '호흡기내과', '흉부외과', '내분비내과', '류마티스내과', '마취통증의학과', '산부인과', '안과', '알레르기내과', '외과', '이비인후과', '재활의학과', '정형외과', '피부과', '병리과', '비뇨기과', '성형외과', '이식혈관외과', '진단검사의학과', '치과', '신종플루거점병원', '핵의학과']


In [ ]:
model.save('/content/drive/MyDrive/PretrainModels/kor_w2v_final')

In [ ]:
train_data.to_csv('/content/drive/MyDrive/PretrainModels/w2v_query_data_final.csv')

In [ ]:
wordrank_extractor = KRWordRank(min_count=2, max_length=20, verbose=True)
beta = 0.9    # PageRank의 decaying factor beta
max_iter = 100
keywords, rank, graph = wordrank_extractor.extract(data_result['symptom'], beta, max_iter)
for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:500]:
  print('%8s:\t%.4f' % (word, r))

# Test Loading model, data

In [ ]:
def tokenizer(sentence):
  tmp = []
  tokenized = okt.morphs(sentence, stem=True) # 토큰화
  stopwords_removed = [word for word in tokenized if not word in stopwords] # 불용어 제거
  changed = [word if word != '귀가' else '귀' for word in stopwords_removed]
  tmp.append(changed)
  return tmp

In [ ]:
query_data = pd.read_csv('/content/drive/MyDrive/PretrainModels/w2v_query_data_final.csv')

In [ ]:
query_data = query_data.set_index('department_single')

In [ ]:
w2v_model = gensim.models.Word2Vec.load('/content/drive/MyDrive/PretrainModels/kor_w2v_final')

In [ ]:
def test_predict(sentence):
  word_list = data_tokenizer(sentence)
  tmp = []
  result = {}
  for word in word_list[0]:
    tmp.append(word)
    try:
        tmp = tmp + [i for i, v in w2v_model.wv.most_similar(word)[:]]
    except:
        continue
  print(tmp)
  for i in train_data.index:
    result[i] = len(set(query_data['symptom'][i]).intersection(set(tmp)))
  return dict(sorted(result.items(), key=operator.itemgetter(1), reverse=True))

In [ ]:
print(test_predict('뒷골이 땡긴다'))

['뒷', '흥분하다', '베개', '사경이', '망치', '베다', '쏘다', '결리다', '딱하다', '짓누르다', '휘청거리다', '골', '중족골', '삼지', '과지', '종축', '무지', '합', '중위', '거대하다', '삼각', '단축', '땡기다']
{'가정의학과': 3, '내분비내과': 3, '마취통증의학과': 3, '소아청소년과': 3, '소화기내과': 3, '순환기내과': 3, '신경과': 3, '신경외과': 3, '외과': 3, '이비인후과': 3, '재활의학과': 3, '정형외과': 3, '감염내과': 2, '내과': 2, '류마티스내과': 2, '방사선종양학과': 2, '산부인과': 2, '성형외과': 2, '신장내과': 2, '안과': 2, '응급의학과': 2, '정신건강의학과': 2, '치과': 2, '피부과': 2, '혈액종양내과': 2, '호흡기내과': 2, '흉부외과': 2, '비뇨기과': 1, '알레르기내과': 1, '이식혈관외과': 1, '진단검사의학과': 1, '병리과': 0, '신종플루거점병원': 0, '핵의학과': 0}
